In [9]:
import spacy
import numpy as np
import pandas as pd
import nltk
import torch
from transformers import pipeline, BartForConditionalGeneration, BartTokenizer
from sklearn.model_selection import train_test_split
nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\USER/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
!pip install torch

In [11]:
!pip install transformers

In [12]:
import pandas as pd

df = pd.read_csv(r"C:\Users\USER\Downloads\data_all_raw_500.csv", on_bad_lines='skip')
  # Deprecated in newer pandas versions, use `on_bad_lines='skip'`
df.head()


,,text;summary
Guingamp defender Benjamin Angoua took an unorthodox method to try and stop his goalkeeper getting sent off. Stopper Mamadou Samassa was about to be given his marching order during the club's Ligue 1 clash against Montpellier for his furious protests after conceding a goal. But as referee Philippe Kalt was brandishing the red card,the Ivorian international knocked it out of the official's hand not once but twice. On the second attempt Angoua succeeded but after fumbling the card,Kalt dismissed Samassa. Angoua fortunately es...
Ahead of another weekend of the Barclays Premier League,Sportsmail brings you the latest squad news,odds and stats on every top flight fixture as...
San Lorenzo will play Real Madrid in Saturday's FIFA World Club final after the Argentine side beat Auckland City 2-1 in extra time on Wednesday night. Pablo Barrientos fired San Lorenzo into the lead just before half time before Angel Berlanga equalised for Auckland City in the 67th minute. But,Mauro Matos pounced early on in the first half of extra time to put the Argentines ahead again. A place in the final against the Galacticos now awaits San Lorenzo with the finale due to take place on Saturday night in Morocco.;Pablo Barrientos scores for San Lorenzo just before the break . Angel Berlanga equalises for Auckland City in the second half . Mauro Matos strikes in extra time to give San Lorenzo 2-1 lead . San Lorenzo will play Real Madrid in Saturday's final. . Auckland beat Setif in quarter-finals,while Argentine side received a bye . Live st...
Angel di Maria has given his first interview as a Manchester United player following his £60million British transfer record switch from Real Madrid. The Argentina star has given United a boost during a troubled week in which they lost 4-0 to League One MK Dons. He has also been getting familiar with his new surroundings,being pictured out and about in Manchester this week. He - and the United fans - will hope he is in the squad to face Burnley on Saturday. Like our dedicated Manchester United Facebook page. Putting their shirt on him: Mancheter United's British record signing Angel di Maria VIDEO I will help United return to the top - Di Maria;Angel Di Maria joined Manchester United on Tuesday in a British transfer record smashing £60m move . The Argentina winger's arrival has given Man Utd a boost amid a poor start to the season . Di Maria has been unveiled today . Man United fans will hope to see him against Burnley on Saturday . Di Maria pictured out in Manchester this week . Click here to watch the interview on MUTV . For more news on United's summer signings click here,NaN
"Hull's Ahmed Elmohamady has pulled out of Egypt's forthcoming African Nations Cup double header. Elmohamady, the right winger or full-back, was injured during a man-of-the-match performance against Crystal Palace at the weekend and will not be travelling for Friday's game in Botswana or the home clash in Cairo five days later. A short statement from the Tigers read: ""The club can confirm that Ahmed Elmohamady has withdrawn from Egypt's African Cup of Nations qualifiers with a back spasm."" Ahmed Elmohamady tries to win the ball for Hull City during Saturday's 2-0 win on against Crystal Palace The Egyptian wing-back attempts to beat Gael Clichy during the Tigers' 4-2 defeat against Manchester City;Hull City's Ahmed Elmohamady is out of Egypt's African Cup of Nations two qualifiers this month due to back spasms . Egypt play Botswana on Friday then again next Wednesday . Elmohamady suffered the injury in Hull's 2-0 win against Crystal Palace . hull have not confirmed how long the player will be out for",NaN,NaN


In [13]:
import pandas as pd

df = pd.read_csv(r"C:\Users\USER\Downloads\data_all_raw_500.csv", sep="\t", on_bad_lines='skip')  # Try tab-separated
print(df.head())


                                        text;summary
0  Guingamp defender Benjamin Angoua took an unor...
1  Ahead of another weekend of the Barclays Premi...
2  By Jonathan Block Several pornographic images ...
3  San Lorenzo will play Real Madrid in Saturday'...
4  QPR midfielder Leroy Fer will be sidelined for...


In [14]:
import pandas as pd

df = pd.read_csv(r"C:\Users\USER\Downloads\data_all_raw_500.csv", sep=";", on_bad_lines='skip')
print(df.head())


                                                text  \
0  Guingamp defender Benjamin Angoua took an unor...   
1  Ahead of another weekend of the Barclays Premi...   
2  By Jonathan Block Several pornographic images ...   
3  San Lorenzo will play Real Madrid in Saturday'...   
4  QPR midfielder Leroy Fer will be sidelined for...   

                                             summary  
0  Ivorian defender Benjamin Angoua nicked the re...  
1  Table toppers Chelsea face a trip to Swansea o...  
2  Sarah Bajc, girlfriend of passenger Philip Woo...  
3  Pablo Barrientos scores for San Lorenzo just b...  
4  QPR won for the first time away this season at...  


In [15]:
print(df.isnull().sum())


text       0
summary    0
dtype: int64


In [16]:
print(df.shape)  


(132, 2)


In [17]:
print(df.duplicated().sum())  # Count duplicate rows


2


In [18]:
df = df.drop_duplicates()


In [19]:
print(df.duplicated().sum())  # Should return 0
print(df.shape)  # Rows should decrease if duplicates were present


0
(130, 2)


In [20]:
import spacy
!python -m spacy download en_core_web_sm



     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 1.7 MB/s eta 0:00:08
     --- ------------------------------------ 1.0/12.8 MB 1.7 MB/s eta 0:00:07
     ---- ----------------------------------- 1.3/12.8 MB 1.7 MB/s eta 0:00:07
     ----- ---------------------------------- 1.8/12.8 MB 1.7 MB/s eta 0:00:07
     ------ --------------------------------- 2.1/12.8 MB 1.7 MB/s eta 0:00:07
     ------- -------------------------------- 2.4/12.8 MB 1.7 MB/s eta 0:00:07
     --------- ------------------------------ 2.9/12.8 MB 1.7 MB/s eta 0:00:06
     --------- ------------------------------ 3.1/12.8 MB 1.7 MB/s eta 0:00:06
     ----------- ---------------------------- 3.7/12.8 MB 1.7 MB/s eta 0:00:06
     ----------- ---------------------------- 3.7/12.8 MB 1.7 MB/s eta 0:00:06
     ------------- -------------------------- 4.2/12.8 MB 1.7 MB/s

In [21]:
import re
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

nlp = spacy.load("en_core_web_sm")
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    words = text.split()
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    return " ".join(words)

df["cleaned_text"] = df["text"].apply(preprocess_text)

print(df.head())  # Check cleaned text


                                                text  \
0  Guingamp defender Benjamin Angoua took an unor...   
1  Ahead of another weekend of the Barclays Premi...   
2  By Jonathan Block Several pornographic images ...   
3  San Lorenzo will play Real Madrid in Saturday'...   
4  QPR midfielder Leroy Fer will be sidelined for...   

                                             summary  \
0  Ivorian defender Benjamin Angoua nicked the re...   
1  Table toppers Chelsea face a trip to Swansea o...   
2  Sarah Bajc, girlfriend of passenger Philip Woo...   
3  Pablo Barrientos scores for San Lorenzo just b...   
4  QPR won for the first time away this season at...   

                                        cleaned_text  
0  guingamp defender benjamin angoua took unortho...  
1  ahead another weekend barclays premier league ...  
2  jonathan block several pornographic images uns...  
3  san lorenzo play real madrid saturdays fifa wo...  
4  qpr midfielder leroy fer sidelined ten weeks s..

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
train_texts, test_texts, train_summaries, test_summaries = train_test_split(
    df["cleaned_text"], df["summary"], test_size=0.2, random_state=42
)

print(f"Training Samples: {len(train_texts)}")
print(f"Testing Samples: {len(test_texts)}")


Training Samples: 104
Testing Samples: 26


In [23]:
from heapq import nlargest

def extractive_summary(text, num_sentences=3):
    doc = nlp(text)
    sentence_scores = {}
    
    for sent in doc.sents:
        for word in sent:
            if word.text.lower() not in stop_words:
                sentence_scores[sent] = sentence_scores.get(sent, 0) + 1
    
    summary_sentences = nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    return " ".join([sent.text for sent in summary_sentences])

df["extractive_summary"] = df["cleaned_text"].apply(lambda x: extractive_summary(x, 3))

print(df[["text", "extractive_summary"]].head())  # Compare summaries


                                                text  \
0  Guingamp defender Benjamin Angoua took an unor...   
1  Ahead of another weekend of the Barclays Premi...   
2  By Jonathan Block Several pornographic images ...   
3  San Lorenzo will play Real Madrid in Saturday'...   
4  QPR midfielder Leroy Fer will be sidelined for...   

                                  extractive_summary  
0  guingamp defender benjamin angoua took unortho...  
1  ahead another weekend barclays premier league ...  
2  jonathan block several pornographic images uns...  
3  fifa world club final argentine side beat auck...  
4  qpr midfielder leroy fer sidelined ten weeks s...  


In [24]:
def abstractive_summary(text, max_length=50, min_length=20):
    if len(text.split()) > 512:  # Reduce input length
        text = " ".join(text.split()[:512])
    
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs["input_ids"], max_length=max_length, min_length=min_length, length_penalty=2.0
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [25]:
import os
os.environ["TRANSFORMERS_CACHE"] = "./models"



In [26]:
!pip install --upgrade transformers


In [27]:
!pip install sumy


In [28]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\USER/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
import nltk
nltk.data.path.append("C:\\Users\\USER\\nltk_data")


In [30]:
import nltk
nltk.download('punkt', force=True)


[nltk_data] Downloading package punkt to C:\Users\USER/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [31]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

def summarize_text(text, num_sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, num_sentences)
    
    return " ".join(str(sentence) for sentence in summary)

# Example usage
text = "Your long paragraph goes here..."
summary = summarize_text(text)
print("Summary:", summary)


Summary: Your long paragraph goes here...


In [32]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

def summarize_text(text, num_sentences=2):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, num_sentences)
    
    return " ".join(str(sentence) for sentence in summary)

# Example usage
text = """Machine learning is a method of data analysis that automates analytical model building. 
Using algorithms that iteratively learn from data, machine learning allows computers to find hidden 
insights without being explicitly programmed. It is widely used in various applications such as 
image recognition, speech recognition, and medical diagnosis. The demand for AI and ML skills 
has increased significantly in the job market."""
summary = summarize_text(text, num_sentences=2)
print("Summary:", summary)


Summary: Machine learning is a method of data analysis that automates analytical model building. Using algorithms that iteratively learn from data, machine learning allows computers to find hidden insights without being explicitly programmed.
